# Common Methods

In [ ]:
import requests
import uuid
import time
import sys

base_url = 'https://app.auger.ai/api/v1'

def get_method(url, json={}):
    payload = {'token': token}
    payload.update(json)
    return requests.get(url,
        headers={
            'Accept': 'application/json, text/plain, */*',
            'Content-Type': 'application/json'
        },
        json= payload
    )

def post_method(url, json={}):
    payload = {'token': token}
    payload.update(json)
    return requests.post(url,
        headers={
            'Accept': 'application/json, text/plain, */*',
            'Content-Type': 'application/json'
        },
        json= payload
    )

def put_method(url, id, json={}):
    payload = {'token': token}
    payload.update(json)
    url = url + "/" + id
    return requests.put(url,
        headers={
            'Accept': 'application/json, text/plain, */*',
            'Content-Type': 'application/json'
        },
        json= payload
    )

# Getting your API token


In [ ]:

token_url = base_url + '/tokens'
email = 'my_emal'
password = 'my_password'
resp = requests.post(token_url,
                     headers={
                        'Accept': 'application/json, text/plain, */*',
                        'Content-Type': 'application/json'
                     },
                     json=  {'email': email, 'password': password}
                    )
                    
token = resp.json()['data']['token']
print(token)

# Deploying a project to a cluster


### First lets get our organization id. 

In [ ]:
orgs_url = base_url + '/organizations'
org_name = 'your organization name'
orgs_json = {'limit': 100,'name': org_name}
resp = get_method(orgs_url, orgs_json).json()
organizations = resp['data']

org_id = organizations[0]['id']
print(org_id)

### Next lets get a list of cluster types to choose from

In [ ]:
instance_types_url = base_url + '/instance_types'
resp = get_method(instance_types_url).json()
instance_types = resp['data']

for instance_type in instance_types:
    print("id: {}    description: {} ".format(instance_type['id'],instance_type['description']))


### Create a project

In [ ]:
projects_url = base_url + '/projects'
projects_json = {'name':'APIProject2', 'organization_id': org_id}
resp = post_method(projects_url, projects_json).json()
print(resp)
project_id = resp['data']['id']

### Or Get an existing project

In [ ]:
projects_url = base_url + '/projects'
name = 'my project name'
projects_json = {'name': name, 'organization_id': org_id}
resp = get_method(projects_url, projects_json).json()
print(resp)
project_id = resp['data'][0]['id']

### Create a cluster and deploy your project

In [ ]:
clusters_url = base_url + '/clusters'
project_id = project_id
clusters_json = {'organization_id': org_id, 'instance_type': 'c5.xlarge', 'project_id': project_id }
resp = post_method(clusters_url, clusters_json).json()
cluster = resp['data']
print(cluster)


### Check cluster status 

In [ ]:
cluster_id = str(cluster['id'])
resp = get_method(clusters_url + '/' + cluster_id, clusters_json).json()
cluster = resp['data']

while cluster['status'] != 'running':
    resp = get_method(clusters_url + '/' + cluster_id, clusters_json).json()
    cluster = resp['data']
    print(cluster['status'])
    time.sleep(30)


# Running an experiment

### First lets upload our dataset 

In [ ]:
project_file_url = projects_url + '/' + str(project_id) + '/files'
dataset_url = 'remote path to data.csv'
project_file_json = { 'name': 'IrisData', 'url': dataset_url, 'file_name': 'iris_data.csv' }
resp = post_method(project_file_url, project_file_json).json()
print(resp)
project_file = resp['data']
project_file_id = project_file['id']
print(project_file)


### Get a project_file

In [ ]:
project_file_url = projects_url + '/' + str(project_id) + '/files'
resp = get_method(project_file_url).json()
#print(resp)
project_files = resp['data']
project_file_id = project_files[0]['id']
#print(project_files)

print(project_file_id)



### Create experiment for dataset

In [ ]:
experiments_url = base_url + '/experiments'
exp_name = "my experiment name"
experiments_json_post = {'project_id': project_id, 'name': exp_name, 'project_file_id': project_file_id}
resp = post_method(experiments_url, experiments_json_post).json()
print(resp)
experiment = resp['data']
experiment_id = experiment['id']
print(experiment)

### Create an experiment session

In [ ]:
exp_session_url = base_url + '/experiment_sessions'

features = [
    {'column_name': 'sepal_length'}, 
    {'column_name': 'sepal_width' },
    {'column_name': 'petal_length'}, 
    {'column_name': 'petal_width' }, 
    {'column_name': 'class', 'datatype': 'categorical','isTarget': True}
    ]

model_settings = {
    'scoring': 'accuracy',
    'max_n_trials': 100,
    'features': features
    }
exp_session_json_post = { 'project_id': project_id, 'experiment_id': experiment_id,
                           'model_settings': model_settings, 'model_type': 'classification',
                        }
    
resp = post_method(exp_session_url,exp_session_json_post).json()
print(resp)
experiment_session = resp['data']
experiment_session_id = experiment_session['id']
print(experiment_session)

### Run the experiment session

In [ ]:

exp_session_update = put_method(exp_session_url, 
                                experiment_session_id, 
                                {
                                    'status': 'preprocess' 
                                })

print(exp_session_update.json())

### Watch our leaderboard grow

In [ ]:
import time
trials_url = exp_session_url + '/'+ experiment_session_id + '/trials'

resp = get_method(trials_url).json()
trials = resp['data']


for trial in trials:
    #print(trial)
    print("{} - {}".format(trial['hyperparameter']['algorithm_name'],trial['score_value']))



resp = get_method(exp_session_url + '/' + experiment_session_id).json()
experiment_session = resp['data']
print(experiment_session['status'])
while experiment_session['status'] in ['preprocess','started']:
    time.sleep(5)
    
    resp = get_method(trials_url).json()
    trials = resp['data']
    for trial in trials:
        #print(trial)
        print("{} - {}".format(trial['hyperparameter']['algorithm_name'],trial['score_value']))

    resp = get_method(exp_session_url + '/' + experiment_session_id).json()
    experiment_session = resp['data']
    print(experiment_session['status'])

